Descargar dependencias


In [ ]:
!apt-get install -y clisp
!pip install scikit-fuzzy

print("ok")


Importar las librerias

In [42]:
import clips
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import gradio as gr

 Definición del Sistema Difuso con scikit-fuzzy
 - Definimos las variables difusas

In [43]:
edad = ctrl.Antecedent(np.arange(0, 101, 1), 'edad')
estado_animo = ctrl.Antecedent(np.arange(0, 11, 1), 'estado_animo')
actividad = ctrl.Antecedent(np.arange(0, 11, 1), 'actividad')
recomendacion = ctrl.Consequent(np.arange(0, 11, 1), 'recomendacion')
dia = ctrl.Antecedent(np.arange(0, 7, 1), 'dia')

Funciones de membresía

In [44]:
# funciones de membresía para edad
edad['joven'] = fuzz.trimf(edad.universe, [0, 15, 30])
edad['adulto'] = fuzz.trimf(edad.universe, [25, 40, 60])
edad['mayor'] = fuzz.trimf(edad.universe, [50, 70, 100])

# funciones de membresía para estado de ánimo
estado_animo['triste'] = fuzz.trimf(estado_animo.universe, [0, 2, 4])
estado_animo['neutral'] = fuzz.trimf(estado_animo.universe, [3, 5, 7])
estado_animo['feliz'] = fuzz.trimf(estado_animo.universe, [6, 8, 10])

# funciones de membresía para actividad
actividad['baja'] = fuzz.trimf(actividad.universe, [0, 2, 4])
actividad['media'] = fuzz.trimf(actividad.universe, [3, 5, 7])
actividad['alta'] = fuzz.trimf(actividad.universe, [6, 8, 10])

# funciones de membresía para recomendación (Consecuente)
recomendacion['mala'] = fuzz.trimf(recomendacion.universe, [0, 2, 4])
recomendacion['media'] = fuzz.trimf(recomendacion.universe, [3, 5, 7])
recomendacion['buena'] = fuzz.trimf(recomendacion.universe, [6, 8, 10])

# funciones de membresia para el dia
dia['entre-semana'] = fuzz.trimf(dia.universe, [0, 2, 4])  # Lunes - Jueves
dia['viernes'] = fuzz.trimf(dia.universe, [4, 5, 5])  # Viernes
dia['fin-de-semana'] = fuzz.trimf(dia.universe, [5, 6, 6])  # Sábado - Domingo




Reglas Difusas

In [45]:
rule1 = ctrl.Rule(edad['joven'] & estado_animo['feliz'], recomendacion['buena'])
rule2 = ctrl.Rule(edad['adulto'] & actividad['baja'], recomendacion['mala'])
rule3 = ctrl.Rule(estado_animo['neutral'] & actividad['media'], recomendacion['media'])
rule4 = ctrl.Rule(dia['viernes'] & estado_animo['feliz'], recomendacion['buena'])
rule5 = ctrl.Rule(dia['entre-semana'] & actividad['baja'], recomendacion['mala'])
rule6 = ctrl.Rule(dia['fin-de-semana'] & estado_animo['neutral'], recomendacion['media'])

Sistema Difuso

In [ ]:
sistema_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
sistema = ctrl.ControlSystemSimulation(sistema_ctrl)

print("✅ Sistema difuso corregido y listo para ejecutar.")

Creación del Archivo CLIPS con las Reglas

In [47]:
with open("musica.clp", "w", encoding="utf-8") as f:
    f.write('''
    (deftemplate usuario
      (slot edad-clasificada)
      (slot estado-clasificado)
      (slot actividad)
      (slot hora)
      (slot dia))

    (deffacts generos-disponibles
      (musica rock "https://spotify.com/rock" "https://img.com/rock.jpg")
      (musica pop "https://spotify.com/pop" "https://img.com/pop.jpg")
      (musica jazz "https://spotify.com/jazz" "https://img.com/jazz.jpg")
      (musica electronica "https://spotify.com/electronic" "https://img.com/electronic.jpg")
      (musica indie "https://spotify.com/indie" "https://img.com/indie.jpg")
      (musica reggae "https://spotify.com/reggae" "https://img.com/reggae.jpg"))
    ''')

Cargar CLIPS y Ejecutar el Motor de Inferencia

In [ ]:
env = clips.Environment()
env.load("musica.clp")
env.reset()
env.run()

Desarrollo de la Interfaz Gráfica con Gradio

In [ ]:
import gradio as gr


def recomendar_musica(edad, estado_animo, actividad, dia_texto):
    # Convertir el día de texto a número
    dias_semana = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]

    try:
        dia_numerico = dias_semana.index(dia_texto.lower())  # Convertir texto a número (0-6)
    except ValueError:
        return "Error: Ingresa un día válido (Ej: lunes, martes, etc.)"

    # Pasar los valores al sistema difuso
    sistema.input['edad'] = edad
    sistema.input['estado_animo'] = estado_animo
    sistema.input['actividad'] = actividad
    sistema.input['dia'] = dia_numerico

    sistema.compute()
    recomendacion = sistema.output['recomendacion']

    return f"Recomendación Difusa: {recomendacion}"


demo = gr.Interface(
    fn=recomendar_musica,
    inputs=[
        gr.Number(label="Edad"),
        gr.Number(label="Estado de Ánimo"),
        gr.Number(label="Actividad"),
        gr.Textbox(label="Día de la Semana")
    ],
    outputs=gr.Textbox(label="Recomendación")
)


demo.launch()
